* Anaconda
  * Out of the box modules
  * Environments
  * Spyder
* Juypter Project
  * Formerly IPython Notebooks
  * Better support for non python kernels
  * Navigation
  * Tips and tricks
* Blaze
  * Data transfer
  * Querying
* Pandas
  * Series
  * DataFrame
  * Input functions
  * Boolean Indexing
  * Groupby and Pivot
  * Merge
  * Map and Apply
* Bokeh
  * Interactive data visualizations
  * Python -> Javascript